In [1]:
import pandas

In [2]:
open_payments = pandas.read_csv('../../data/processed/open_payments.csv', index_col=0)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
all_doctors = pandas.read_csv('../../data/processed/all.csv', index_col=0)

In [4]:
apply_func = lambda x: x.upper() if isinstance(x, str) else None

open_payments['physician_first_name'] = open_payments['physician_first_name'].apply(apply_func)
open_payments['physician_last_name' ] = open_payments['physician_last_name' ].apply(apply_func)

all_doctors['first_name'] = all_doctors['first_name'].apply(apply_func)
all_doctors['last_name' ] = all_doctors['last_name' ].apply(apply_func)

In [5]:
all_doctors = all_doctors\
    .groupby(['first_name', 'last_name', 'state'])\
    .first()\
    .reset_index()

In [6]:
merged = pandas.merge(
    open_payments, 
    all_doctors.reset_index(), # preserve individual doctors in column `index`
    how='right',
    left_on=['physician_first_name', 'physician_last_name'],#, 'specialty_code'],
    right_on=['first_name', 'last_name'],#, 'specialty_code']
    suffixes=['_open_payments', None]
)

In [7]:
# how many unique doctors take money?
for source in all_doctors['src'].unique():
    individual_doctors_taking_money = merged.groupby('physician_profile_id').first()
    likely_prescribers_count = len(
        all_doctors[(all_doctors['src'] == source)]
    )
    on_take = len(
        individual_doctors_taking_money[
            (individual_doctors_taking_money['src'] == source)
            
        ]
    )
    on_take_pct = on_take / likely_prescribers_count
    print(f"For data from {source}, % of doctors on take is {round(on_take_pct * 100, 5)}% out of {likely_prescribers_count}")

For data from asoprs, % of doctors on take is 42.28769% out of 577
For data from endocrinologists, % of doctors on take is 18.10437% out of 939
For data from tepezza, % of doctors on take is 65.74468% out of 470


In [8]:
merged.to_csv('../../data/processed/all_transactions.csv')

In [9]:
total_money = merged\
    .groupby('physician_profile_id')\
    .sum()\
    ['total_amount_of_payment_usdollars']

In [10]:
dates = merged\
    .groupby('physician_profile_id')\
    ['date_of_payment']\
    .apply(','.join)
    

In [11]:
# get grouped paid doctors and all unpaid doctors as one
agg = merged\
    .groupby('physician_profile_id')\
    .first()
agg['total_money'] = total_money
agg['dates_of_payments'] = dates
agg = agg\
    .reset_index()\
    .append(
        merged[pandas.isna(merged['physician_profile_id'])]
    )\
    .drop(columns=['index', 'total_amount_of_payment_usdollars', 'date_of_payment'])

In [12]:
agg.to_csv('/Users/eab06/Desktop/WJB/PythonProjects/HT_Data/data/processed/aggregated.csv')